In [75]:
import numpy as np
import pandas as pd
import pyarrow as pa
import matplotlib.pyplot as plt

In [116]:
df = pd.read_csv('2019_-_2020_School_Locations.csv', keep_default_na=True, delimiter=',', skipinitialspace=True)

In [117]:
df.shape

(2190, 41)

In [118]:
df.head(5)

,fiscal_year,system_code,location_code,location_name,BEDS,Managed_by_name,location_type_description,Location_Category_Description,Grades_text,Grades_final_text,...,Administrative_District_Name,community_school_sup_name,Tier_3_Support_Location_Name,Tier_3_Support_Leader_Name,Tier_2_Support_Location_Name,HighSchool_Network_Location_Code,HighSchool_Network_Name,HighSchool_Network_Superintendent,Community_district 1,Police_precinct
0,2020,15K001,K001,P.S. 001 The Bergen,331500010001,DOE,General Academic,Elementary,"PK,0K,01,02,03,04,05,SE","PK,0K,01,02,03,04,05",...,COMMUNITY SCHOOL DISTRICT 15,"SKOP, ANITA",NYCDOE Borough Office - Brooklyn North,NaN,School Support Team 5- Brooklyn North,NaN,NaN,NaN,307.0,72.0
1,2020,17K002,K002,Parkside Preparatory Academy,331700010002,DOE,General Academic,Junior High-Intermediate-Middle,"06,07,08,SE","06,07,08",...,COMMUNITY SCHOOL DISTRICT 17,"ELLIS, CLARENCE",NYCDOE Borough Office - Brooklyn South,Mauriciere de Govia,School Support Team 2- Brooklyn South,NaN,NaN,NaN,309.0,71.0
2,2020,13K003,K003,P.S. 003 The Bedford Village,331300010003,DOE,General Academic,Elementary,"PK,0K,01,02,03,04,05,SE","PK,0K,01,02,03,04,05",...,COMMUNITY SCHOOL DISTRICT 13,"SAMUELS, KAMAR",NYCDOE Borough Office - Brooklyn North,NaN,School Support Team 3- Brooklyn North,NaN,NaN,NaN,303.0,79.0
3,2020,75K004,K004,P.S. K004,307500013004,DOE,Special Education,Elementary,"PK,0K,01,02,03,04,05,SE","PK,0K,01,02,03,04,05,06,07,SE",...,CITYWIDE SPECIAL EDUCATION,"LOUISSAINT, KETLER",D75 CITYWIDE BCO,Tillman Roberto,Children First Network 752,NaN,NaN,NaN,305.0,75.0
4,2020,16K005,K005,P.S. 005 Dr. Ronald McNair,331600010005,DOE,General Academic,Elementary,"PK,0K,01,02,03,04,05,SE","PK,0K,01,02,03,04,05",...,COMMUNITY SCHOOL DISTRICT 16,"MARTIN, YOLANDA",NYCDOE Borough Office - Brooklyn North,NaN,School Support Team 4- Brooklyn North,NaN,NaN,NaN,303.0,81.0


In [119]:
# check duplicated rows
print('Number of duplicate (excluding original) rows is:', df.duplicated().sum())
print('Number of duplicate rows (including first) in the table is:', df[df.duplicated(keep=False)].shape[0])
# Show duplicate row data that can be dropped
df[df.duplicated(keep=False)]

Number of duplicate (excluding original) rows is: 59
Number of duplicate rows (including first) in the table is: 118


,fiscal_year,system_code,location_code,location_name,BEDS,Managed_by_name,location_type_description,Location_Category_Description,Grades_text,Grades_final_text,...,Administrative_District_Name,community_school_sup_name,Tier_3_Support_Location_Name,Tier_3_Support_Leader_Name,Tier_2_Support_Location_Name,HighSchool_Network_Location_Code,HighSchool_Network_Name,HighSchool_Network_Superintendent,Community_district 1,Police_precinct
26,2020,16K026,K026,P.S. 026 Jesse Owens,331600010026,DOE,General Academic,Elementary,"PK,0K,01,02,03,04,05,SE","PK,0K,01,02,03,04,05",...,COMMUNITY SCHOOL DISTRICT 16,"MARTIN, YOLANDA",NYCDOE Borough Office - Brooklyn North,NaN,School Support Team 7- Brooklyn North,NaN,NaN,NaN,303.0,81.0
27,2020,16K026,K026,P.S. 026 Jesse Owens,331600010026,DOE,General Academic,Elementary,"PK,0K,01,02,03,04,05,SE","PK,0K,01,02,03,04,05",...,COMMUNITY SCHOOL DISTRICT 16,"MARTIN, YOLANDA",NYCDOE Borough Office - Brooklyn North,NaN,School Support Team 7- Brooklyn North,NaN,NaN,NaN,303.0,81.0
42,2020,23K041,K041,P.S. 041 Francis White,332300010041,DOE,General Academic,K-8,"0K,01,02,03,04,05,06,07,08,SE","0K,01,02,03,04,05,06,07,08",...,COMMUNITY SCHOOL DISTRICT 23,"PATE, MIATHERESA",NYCDOE Borough Office - Brooklyn North,NaN,School Support Team 7- Brooklyn North,NaN,NaN,NaN,316.0,73.0
43,2020,23K041,K041,P.S. 041 Francis White,332300010041,DOE,General Academic,K-8,"0K,01,02,03,04,05,06,07,08,SE","0K,01,02,03,04,05,06,07,08",...,COMMUNITY SCHOOL DISTRICT 23,"PATE, MIATHERESA",NYCDOE Borough Office - Brooklyn North,NaN,School Support Team 7- Brooklyn North,NaN,NaN,NaN,316.0,73.0
65,2020,20K069,K069,P.S. 69 Vincent D. Grippo School,332000010069,DOE,General Academic,Elementary,"0K,01,02,03,04,05,SE","0K,01,02,03,04,05",...,COMMUNITY SCHOOL DISTRICT 20,"COSTANTINO, KARINA",NYCDOE Borough Office - Brooklyn South,Mauriciere de Govia,School Support Team 4- Brooklyn South,NaN,NaN,NaN,310.0,68.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2127,2020,10X575,X575,Norwood Academy Of Communal Excellence At The ...,321000010575,DOE,Budget Only,Junior High-Intermediate-Middle,NaN,NaN,...,COMMUNITY SCHOOL DISTRICT 10,"HULLA, MARIBEL",NaN,NaN,NaN,NaN,NaN,NaN,207.0,52.0
2175,2020,10X696,X696,High School of American Studies at Lehman College,321000011696,DOE,General Academic,High school,"09,10,11,12","09,10,11,12",...,NaN,NaN,NYCDOE Borough Office - Bronx,"SAFFER, NANCY",School Support Team 1- Bronx,HS02,HS Network 02 Superintendent Office,"STAPLE, CARRON",208.0,52.0
2176,2020,10X696,X696,High School of American Studies at Lehman College,321000011696,DOE,General Academic,High school,"09,10,11,12","09,10,11,12",...,NaN,NaN,NYCDOE Borough Office - Bronx,"SAFFER, NANCY",School Support Team 1- Bronx,HS02,HS Network 02 Superintendent Office,"STAPLE, CARRON",208.0,52.0
2188,2020,75X811,X811,P.S. X811,307500012811,DOE,Special Education,Secondary School,"08,09,10,11,12,SE","07,08,09,10,11,12,SE",...,CITYWIDE SPECIAL EDUCATION,"LOUISSAINT, KETLER",D75 CITYWIDE BCO,Tillman Roberto,Children First Network 755,NaN,NaN,NaN,203.0,42.0


In [120]:
# drop duplicates - duplicates might have arised due to the way the data was collected 
df = df.drop_duplicates()
print('Number of duplicate (excluding original) rows is:', df.duplicated().sum())
print('Number of duplicate rows (including first) in the table is:', df[df.duplicated(keep=False)].shape[0])

Number of duplicate (excluding original) rows is: 0
Number of duplicate rows (including first) in the table is: 0


In [121]:
print("Number of rows", df.shape[0])

Number of rows 2131


In [122]:
print("Number of columns", df.shape[1])

Number of columns 41


In [123]:
df = df[['location_name', 'State_code', 'Status_descriptions', 'LONGITUDE', 'LATITUDE']]

In [124]:
df

,location_name,State_code,Status_descriptions,LONGITUDE,LATITUDE
0,P.S. 001 The Bergen,NY,Open,-74.011420,40.648959
1,Parkside Preparatory Academy,NY,Open,-73.951575,40.656423
2,P.S. 003 The Bedford Village,NY,Open,-73.955219,40.682311
3,P.S. K004,NY,Open,-73.879276,40.658500
4,P.S. 005 Dr. Ronald McNair,NY,Open,-73.921970,40.685241
...,...,...,...,...,...
2184,P.S. 723,NY,Open,-73.831856,40.882980
2185,Readnet Bronx Charter School,NY,Closed,-73.917000,40.814900
2186,Bronx Charter School for the Arts,NY,Open,-73.886477,40.821815
2187,J. M. Rapport School Career Development,NY,Open,-73.909363,40.811051


In [125]:
# count how many schools are closed 
closed_count = len(df[df['Status_descriptions'] == 'Closed'])
print("Number of occurrences where 'Status_descriptions' is 'Closed':", closed_count)


Number of occurrences where 'Status_descriptions' is 'Closed': 253


In [126]:
df = df[df['Status_descriptions'] != 'Closed']

In [127]:
count_not_NY = len(df[df['State_code'] != 'NY'])
print(count_not_NY)

0


In [128]:
# no need for other columns now - we know that they are in the new york state , we know that these schools are open
df = df[['location_name', 'LONGITUDE', 'LATITUDE']]

In [129]:
df

,location_name,LONGITUDE,LATITUDE
0,P.S. 001 The Bergen,-74.011420,40.648959
1,Parkside Preparatory Academy,-73.951575,40.656423
2,P.S. 003 The Bedford Village,-73.955219,40.682311
3,P.S. K004,-73.879276,40.658500
4,P.S. 005 Dr. Ronald McNair,-73.921970,40.685241
...,...,...,...
2183,P.S. X721 - Stephen McSweeney School,-73.839114,40.842725
2184,P.S. 723,-73.831856,40.882980
2186,Bronx Charter School for the Arts,-73.886477,40.821815
2187,J. M. Rapport School Career Development,-73.909363,40.811051


In [130]:
without_long_lat = df[df['LONGITUDE'].isnull() | df['LATITUDE'].isnull()]

without_long_lat

,location_name,LONGITUDE,LATITUDE
1382,P.S. Q256,NaN,NaN


In [131]:
# drop the rows without Longitude and Latitude.
df = df.drop(without_long_lat.index)
df.shape

(1877, 3)

In [132]:
# check the data type
df.dtypes

location_name     object
LONGITUDE        float64
LATITUDE         float64
dtype: object

In [133]:
# check null value
df.isnull().sum()

location_name    0
LONGITUDE        0
LATITUDE         0
dtype: int64

In [134]:
df['Industry'] = 'Education'

In [135]:
df

,location_name,LONGITUDE,LATITUDE,Industry
0,P.S. 001 The Bergen,-74.011420,40.648959,Education
1,Parkside Preparatory Academy,-73.951575,40.656423,Education
2,P.S. 003 The Bedford Village,-73.955219,40.682311,Education
3,P.S. K004,-73.879276,40.658500,Education
4,P.S. 005 Dr. Ronald McNair,-73.921970,40.685241,Education
...,...,...,...,...
2183,P.S. X721 - Stephen McSweeney School,-73.839114,40.842725,Education
2184,P.S. 723,-73.831856,40.882980,Education
2186,Bronx Charter School for the Arts,-73.886477,40.821815,Education
2187,J. M. Rapport School Career Development,-73.909363,40.811051,Education


In [136]:
import geopandas as gpd
from shapely.geometry import Point, Polygon

In [137]:
# change 'Longitude' and 'Latitude' into GeoDataFrame
df['Coordinates'] = list(zip(df.LONGITUDE, df.LATITUDE))
df['Coordinates'] = df['Coordinates'].apply(Point)

gdf_Education = gpd.GeoDataFrame(df, geometry='Coordinates')

In [138]:
gdf_Education

,location_name,LONGITUDE,LATITUDE,Industry,Coordinates
0,P.S. 001 The Bergen,-74.011420,40.648959,Education,POINT (-74.01142 40.64896)
1,Parkside Preparatory Academy,-73.951575,40.656423,Education,POINT (-73.95158 40.65642)
2,P.S. 003 The Bedford Village,-73.955219,40.682311,Education,POINT (-73.95522 40.68231)
3,P.S. K004,-73.879276,40.658500,Education,POINT (-73.87928 40.65850)
4,P.S. 005 Dr. Ronald McNair,-73.921970,40.685241,Education,POINT (-73.92197 40.68524)
...,...,...,...,...,...
2183,P.S. X721 - Stephen McSweeney School,-73.839114,40.842725,Education,POINT (-73.83911 40.84273)
2184,P.S. 723,-73.831856,40.882980,Education,POINT (-73.83186 40.88298)
2186,Bronx Charter School for the Arts,-73.886477,40.821815,Education,POINT (-73.88648 40.82182)
2187,J. M. Rapport School Career Development,-73.909363,40.811051,Education,POINT (-73.90936 40.81105)


In [139]:
# Read the geojson file
taxi_zone = gpd.read_file('NYC Taxi Zones.geojson')
taxi_zone

,shape_area,objectid,shape_leng,location_id,zone,borough,geometry
0,0.0007823067885,1,0.116357453189,1,Newark Airport,EWR,"MULTIPOLYGON (((-74.18445 40.69500, -74.18449 ..."
1,0.00486634037837,2,0.43346966679,2,Jamaica Bay,Queens,"MULTIPOLYGON (((-73.82338 40.63899, -73.82277 ..."
2,0.000314414156821,3,0.0843411059012,3,Allerton/Pelham Gardens,Bronx,"MULTIPOLYGON (((-73.84793 40.87134, -73.84725 ..."
3,0.000111871946192,4,0.0435665270921,4,Alphabet City,Manhattan,"MULTIPOLYGON (((-73.97177 40.72582, -73.97179 ..."
4,0.000497957489363,5,0.0921464898574,5,Arden Heights,Staten Island,"MULTIPOLYGON (((-74.17422 40.56257, -74.17349 ..."
...,...,...,...,...,...,...,...
258,0.000168611097013,256,0.0679149669603,256,Williamsburg (South Side),Brooklyn,"MULTIPOLYGON (((-73.95834 40.71331, -73.95681 ..."
259,0.000394552487366,259,0.126750305191,259,Woodlawn/Wakefield,Bronx,"MULTIPOLYGON (((-73.85107 40.91037, -73.85207 ..."
260,0.000422345326907,260,0.133514154636,260,Woodside,Queens,"MULTIPOLYGON (((-73.90175 40.76078, -73.90147 ..."
261,0.0000343423231652,261,0.0271204563616,261,World Trade Center,Manhattan,"MULTIPOLYGON (((-74.01333 40.70503, -74.01327 ..."


In [140]:
gdf_taxi_zone = gpd.GeoDataFrame(taxi_zone, geometry='geometry')
gdf_taxi_zone

,shape_area,objectid,shape_leng,location_id,zone,borough,geometry
0,0.0007823067885,1,0.116357453189,1,Newark Airport,EWR,"MULTIPOLYGON (((-74.18445 40.69500, -74.18449 ..."
1,0.00486634037837,2,0.43346966679,2,Jamaica Bay,Queens,"MULTIPOLYGON (((-73.82338 40.63899, -73.82277 ..."
2,0.000314414156821,3,0.0843411059012,3,Allerton/Pelham Gardens,Bronx,"MULTIPOLYGON (((-73.84793 40.87134, -73.84725 ..."
3,0.000111871946192,4,0.0435665270921,4,Alphabet City,Manhattan,"MULTIPOLYGON (((-73.97177 40.72582, -73.97179 ..."
4,0.000497957489363,5,0.0921464898574,5,Arden Heights,Staten Island,"MULTIPOLYGON (((-74.17422 40.56257, -74.17349 ..."
...,...,...,...,...,...,...,...
258,0.000168611097013,256,0.0679149669603,256,Williamsburg (South Side),Brooklyn,"MULTIPOLYGON (((-73.95834 40.71331, -73.95681 ..."
259,0.000394552487366,259,0.126750305191,259,Woodlawn/Wakefield,Bronx,"MULTIPOLYGON (((-73.85107 40.91037, -73.85207 ..."
260,0.000422345326907,260,0.133514154636,260,Woodside,Queens,"MULTIPOLYGON (((-73.90175 40.76078, -73.90147 ..."
261,0.0000343423231652,261,0.0271204563616,261,World Trade Center,Manhattan,"MULTIPOLYGON (((-74.01333 40.70503, -74.01327 ..."


In [141]:
# Ensure both GeoDataFrames are using the same CRS
gdf_Education = gdf_Education.set_crs("EPSG:4326")
gdf_taxi_zone = gdf_taxi_zone.set_crs("EPSG:4326")

# Perform spatial join with the 'predicate' parameter
result = gpd.sjoin(gdf_Education, gdf_taxi_zone, how="inner", predicate='intersects')

In [142]:
result

,location_name,LONGITUDE,LATITUDE,Industry,Coordinates,index_right,shape_area,objectid,shape_leng,location_id,zone,borough
0,P.S. 001 The Bergen,-74.011420,40.648959,Education,POINT (-74.01142 40.64896),227,0.000993414783816,228,0.177685177706,228,Sunset Park West,Brooklyn
24,P.S. 024,-74.004533,40.652832,Education,POINT (-74.00453 40.65283),227,0.000993414783816,228,0.177685177706,228,Sunset Park West,Brooklyn
52,P.S. K053,-73.988743,40.660805,Education,POINT (-73.98874 40.66081),227,0.000993414783816,228,0.177685177706,228,Sunset Park West,Brooklyn
76,J.H.S. 088 Peter Rouget,-73.988743,40.660805,Education,POINT (-73.98874 40.66081),227,0.000993414783816,228,0.177685177706,228,Sunset Park West,Brooklyn
124,I.S. 136 Charles O. Dewey,-74.007596,40.652127,Education,POINT (-74.00760 40.65213),227,0.000993414783816,228,0.177685177706,228,Sunset Park West,Brooklyn
...,...,...,...,...,...,...,...,...,...,...,...,...
2034,Atmosphere Charter School,-73.910426,40.875161,Education,POINT (-73.91043 40.87516),152,0.0000324684964817,153,0.024737239022,153,Marble Hill,Manhattan
2049,Marble Hill High School for International Studies,-73.912686,40.877379,Education,POINT (-73.91269 40.87738),152,0.0000324684964817,153,0.024737239022,153,Marble Hill,Manhattan
2092,New Visions Charter High School for Advanced Math,-73.912686,40.877379,Education,POINT (-73.91269 40.87738),152,0.0000324684964817,153,0.024737239022,153,Marble Hill,Manhattan
2099,Bronx Theatre High School,-73.912686,40.877379,Education,POINT (-73.91269 40.87738),152,0.0000324684964817,153,0.024737239022,153,Marble Hill,Manhattan


In [143]:
result = result[['location_name', 'LONGITUDE', 'LATITUDE', 'Industry', 'borough', 'zone','location_id']]
result

,location_name,LONGITUDE,LATITUDE,Industry,borough,zone,location_id
0,P.S. 001 The Bergen,-74.011420,40.648959,Education,Brooklyn,Sunset Park West,228
24,P.S. 024,-74.004533,40.652832,Education,Brooklyn,Sunset Park West,228
52,P.S. K053,-73.988743,40.660805,Education,Brooklyn,Sunset Park West,228
76,J.H.S. 088 Peter Rouget,-73.988743,40.660805,Education,Brooklyn,Sunset Park West,228
124,I.S. 136 Charles O. Dewey,-74.007596,40.652127,Education,Brooklyn,Sunset Park West,228
...,...,...,...,...,...,...,...
2034,Atmosphere Charter School,-73.910426,40.875161,Education,Manhattan,Marble Hill,153
2049,Marble Hill High School for International Studies,-73.912686,40.877379,Education,Manhattan,Marble Hill,153
2092,New Visions Charter High School for Advanced Math,-73.912686,40.877379,Education,Manhattan,Marble Hill,153
2099,Bronx Theatre High School,-73.912686,40.877379,Education,Manhattan,Marble Hill,153


In [144]:
result.isnull().sum()

location_name    0
LONGITUDE        0
LATITUDE         0
Industry         0
borough          0
zone             0
location_id      0
dtype: int64

# the code below used for model

In [145]:
result = result[['Industry', 'location_id', 'location_name']]
result

,Industry,location_id,location_name
0,Education,228,P.S. 001 The Bergen
24,Education,228,P.S. 024
52,Education,228,P.S. K053
76,Education,228,J.H.S. 088 Peter Rouget
124,Education,228,I.S. 136 Charles O. Dewey
...,...,...,...
2034,Education,153,Atmosphere Charter School
2049,Education,153,Marble Hill High School for International Studies
2092,Education,153,New Visions Charter High School for Advanced Math
2099,Education,153,Bronx Theatre High School


In [149]:
df_combined = result.groupby(['location_id', 'Industry']).size().reset_index(name='Counts')

df_combined

,location_id,Industry,Counts
0,10,Education,8
1,101,Education,6
2,102,Education,3
3,103,Education,1
4,106,Education,1
...,...,...,...
218,92,Education,11
219,94,Education,12
220,95,Education,12
221,97,Education,12


In [150]:
df_combined.columns = df_combined.columns.str.replace('location_id', 'Taxi_zone')
df_combined.columns = df_combined.columns.str.replace('Industry', 'Educational Facility')

In [151]:
df_combined

,Taxi_zone,Educational Facility,Counts
0,10,Education,8
1,101,Education,6
2,102,Education,3
3,103,Education,1
4,106,Education,1
...,...,...,...
218,92,Education,11
219,94,Education,12
220,95,Education,12
221,97,Education,12


In [152]:
# check the outcome is correct
total_count = df_combined['Counts'].sum()
total_count

1877

In [153]:
df_combined.to_csv("basic_EducationData.csv", index=False)